# Jigsaw 2017 - RoBERTa BCE

## Variables d'environnement

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Id des GPU disponibles : 0 et 1

## Importation

In [2]:
import os
import time
import datetime
from typing import Any, Union, Dict, List
import uuid
import json

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torchtext
import nltk
import sklearn
import transformers
import torchmetrics as tm
from torchmetrics import MetricCollection, Metric, Accuracy, Precision, Recall, AUROC, HammingDistance, F1Score, ROC, AUC, PrecisionRecallCurve


from loguru import logger
from tqdm.auto import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

## Constantes

In [3]:
CUSTOME_NAME = "jigsaw2017-roberta-bce"

# Dataset
DATA_DIR_PATH = os.path.abspath("../../data")
TRAIN_DATASET_PATH = os.path.join(DATA_DIR_PATH, "jigsaw2017-train.csv")
TEST_DATASET_PATH = os.path.join(DATA_DIR_PATH, "jigsaw2017-test.csv")
VALID_DATASET_PATH = os.path.join(DATA_DIR_PATH, "jigsaw2017-valid.csv")
LABEL_LIST = ['toxic', 'severe_toxic', 'obscene', 'threat',
            'insult', 'identity_hate']

# Session
SESSION_DIR_PATH = os.path.abspath("../../session")
SESSION_DATETIME = datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S-%f")
SESSION_NAME = f"{CUSTOME_NAME}_{SESSION_DATETIME}"
CURRENT_SESSION_DIR_PATH = os.path.join(SESSION_DIR_PATH, SESSION_NAME)
# Créer le dossier de la session
os.makedirs(CURRENT_SESSION_DIR_PATH, exist_ok=True)

# Architecture de fichier dans `CURRENT_SESSION_DIR_PATH`
LOG_FILE_NAME = f"{SESSION_NAME}.loguru.log"
MODEL_FILE_NAME = f"{SESSION_NAME}.model"
TEST_FILE_NAME = f"{SESSION_NAME}.test.csv"
METRIC_FILE_NAME = f"{SESSION_NAME}.metric.json"
LOG_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, LOG_FILE_NAME)
MODEL_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, MODEL_FILE_NAME)
TEST_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, TEST_FILE_NAME)
METRIC_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, METRIC_FILE_NAME)

# CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Logging

In [4]:
logger.add(LOG_FILE_PATH, level="TRACE")
logger.info(f"{SESSION_NAME=}")
logger.info(f"{TRAIN_DATASET_PATH=}")
logger.info(f"{TEST_DATASET_PATH=}")
logger.info(f"{CURRENT_SESSION_DIR_PATH=}")
logger.info(f"{LABEL_LIST=}")

2022-03-28 18:23:00.675 | INFO     | __main__:<cell line: 2>:2 - SESSION_NAME='jigsaw2017-roberta-bce_2022-03-28T18-23-00-343919'
2022-03-28 18:23:00.677 | INFO     | __main__:<cell line: 3>:3 - TRAIN_DATASET_PATH='/work2/home/ing1/corentin/hatespeech-detection-models/data/jigsaw2017-train.csv'
2022-03-28 18:23:00.678 | INFO     | __main__:<cell line: 4>:4 - TEST_DATASET_PATH='/work2/home/ing1/corentin/hatespeech-detection-models/data/jigsaw2017-test.csv'
2022-03-28 18:23:00.680 | INFO     | __main__:<cell line: 5>:5 - CURRENT_SESSION_DIR_PATH='/work2/home/ing1/corentin/hatespeech-detection-models/session/jigsaw2017-roberta-bce_2022-03-28T18-23-00-343919'
2022-03-28 18:23:00.681 | INFO     | __main__:<cell line: 6>:6 - LABEL_LIST=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


## Vérifier la cohérence de l'architecture et l'accès aux ressources

In [5]:
logger.info(f"Checking consistency...")

# Vérifier l'accès aux datasets
if not os.path.exists(TRAIN_DATASET_PATH):
    logger.critical(f"Train dataset does not exist !")
    raise RuntimeError("Train dataset does not exist !")
if not os.path.exists(TEST_DATASET_PATH):
    logger.critical(f"Test dataset does not exist !")
    raise RuntimeError("Test dataset does not exist !")
logger.success("Datasets are reachable")

# Vérifier l'accès aux GPU
GPU_IS_AVAILABLE = torch.cuda.is_available()
GPU_COUNT = torch.cuda.device_count()
logger.info(f"{GPU_IS_AVAILABLE=}")
logger.info(f"{GPU_COUNT=}")
if not GPU_IS_AVAILABLE:
    logger.critical("GPU and CUDA are not available !")
    raise RuntimeError("GPU and CUDA are not available !")
logger.success("GPU and CUDA are available")
logger.info(f"{device=}")
for gpu_id in range(GPU_COUNT):
    gpu_name = torch.cuda.get_device_name(0)
    logger.info(f"GPU {gpu_id} : {gpu_name}")

2022-03-28 18:23:00.995 | INFO     | __main__:<cell line: 1>:1 - Checking consistency...
2022-03-28 18:23:00.997 | SUCCESS  | __main__:<cell line: 10>:10 - Datasets are reachable
2022-03-28 18:23:00.998 | INFO     | __main__:<cell line: 15>:15 - GPU_IS_AVAILABLE=True
2022-03-28 18:23:00.999 | INFO     | __main__:<cell line: 16>:16 - GPU_COUNT=1
2022-03-28 18:23:01.001 | SUCCESS  | __main__:<cell line: 20>:20 - GPU and CUDA are available
2022-03-28 18:23:01.002 | INFO     | __main__:<cell line: 21>:21 - device=device(type='cuda')
2022-03-28 18:23:01.005 | INFO     | __main__:<cell line: 22>:24 - GPU 0 : NVIDIA TITAN X (Pascal)


## Dataset

In [6]:
all_train_df = pd.read_csv(TRAIN_DATASET_PATH, index_col=0)
logger.success("Dataset loaded !")

2022-03-28 18:23:02.123 | SUCCESS  | __main__:<cell line: 2>:2 - Dataset loaded !


In [7]:
train_df = all_train_df
validation_df = pd.read_csv(VALID_DATASET_PATH, index_col=0)

In [8]:
class JigsawDataset(Dataset):
    def __init__(self, data_df, tokenizer):
        self.data = data_df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        comment = self.data.iloc[index]["comment_text"]
        label = torch.tensor(self.data.iloc[index][LABEL_LIST].tolist(), dtype=torch.float)
        
        token_list, attention_mask = self.text_to_token_and_mask(comment)

        return dict(index=index, ids=token_list, mask=attention_mask, labels=label)
    
    def text_to_token_and_mask(self, input_text):
        tokenization_dict = tokenizer.encode_plus(input_text,
                                add_special_tokens=True,
                                max_length=128,
                                padding='max_length',
                                truncation=True,
                                return_attention_mask=True,
                                return_tensors='pt')
        token_list = tokenization_dict["input_ids"].flatten()
        attention_mask = tokenization_dict["attention_mask"].flatten()
        return (token_list, attention_mask)

## Model

In [9]:
def set_lr(optim, lr):
    '''
    Set the learning rate in the optimizer
    '''
    for g in optim.param_groups:
        g['lr'] = lr
    return optim

In [10]:
# Transformer class and functions for models and predictions

class TransformerClassifierStack(nn.Module):
    def __init__(self, tr_model, nb_labels, dropout_prob=0.4, freeze=False):
        super().__init__()
        self.tr_model = tr_model

        # Stack features of 4 last encoders
        self.hidden_dim = tr_model.config.hidden_size * 4

        # hidden linear for the classification
        self.dropout = nn.Dropout(dropout_prob)
        self.hl = nn.Linear(self.hidden_dim, self.hidden_dim)

        # Last Linear for the classification
        self.last_l = nn.Linear(self.hidden_dim, nb_labels)

        # freeze all the parameters if necessary
        for param in self.tr_model.parameters():
            param.requires_grad = not freeze

        # init learning params of last layers
        torch.nn.init.xavier_uniform_(self.hl.weight)
        torch.nn.init.xavier_uniform_(self.last_l.weight)

    def forward(self, ids, mask):
        # ids = [batch_size, padded_seq_len]
        # mask = [batch_size, padded_seq_len]
        # mask: avoid to make self attention on padded data
        tr_output = self.tr_model(input_ids=ids,
                                  attention_mask=mask,
                                  output_hidden_states=True)

        # Get all the hidden states
        hidden_states = tr_output['hidden_states']

        # hs_* = [batch_size, padded_seq_len, 768]
        hs_1 = hidden_states[-1][:, 0, :]
        hs_2 = hidden_states[-2][:, 0, :]
        hs_3 = hidden_states[-3][:, 0, :]
        hs_4 = hidden_states[-4][:, 0, :]

        # features_vec = [batch_size, 768 * 4]
        features_vec = torch.cat([hs_1, hs_2, hs_3, hs_4], dim=-1)

        x = self.dropout(features_vec)
        x = self.hl(x)

        # x = [batch_size, 768 * 4]
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.last_l(x)
        x = torch.sigmoid(x)

        # x = [batch_size, 1]
        return x

def load_roberta_model(nb_labels):
    '''
    Load RoBERTa model without any checkpoint
    RoBERTa for finetuning
    '''
    logger.info(f"transformers.RobertaTokenizer : roberta-base")
    logger.info(f"transformers.AutoModel : roberta-base")
    tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')
    tr_model = transformers.AutoModel.from_pretrained('roberta-base')
    model = TransformerClassifierStack(tr_model, nb_labels, freeze=True)
    return model, tokenizer


def load_roberta_pretrained(path, nb_labels, lr=2e-5):
    '''
    Load RoBERTa from checkout point (already trained on Hate Speech tasks)
    '''
    tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')
    tr_model = transformers.AutoModel.from_pretrained('roberta-base')
    model = TransformerClassifierStack(tr_model, nb_labels)

    loaded = torch.load(path)
    model.load_state_dict(loaded['state_dict'])

    optimizer = transformers.AdamW(model.parameters(), lr=lr)
    optimizer.load_state_dict(loaded['optimizer_dict'])
    optimizer = set_lr(optimizer, lr)

    return model, tokenizer, optimizer

def preds_fn(batch, model, device):
    '''
    Get the predictions for one batch according to the model
    '''
    b_input = batch['ids'].to(device)
    b_mask = batch['mask'].to(device)

    return model(b_input, b_mask)

In [11]:
# Load the model
model, tokenizer = load_roberta_model(nb_labels=len(LABEL_LIST))
logger.success("Model loaded !")

2022-03-28 18:23:03.640 | INFO     | __main__:load_roberta_model:63 - transformers.RobertaTokenizer : roberta-base
2022-03-28 18:23:03.642 | INFO     | __main__:load_roberta_model:64 - transformers.AutoModel : roberta-base
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022-03-28 18:23:07.993 | SUCCESS  | __main__:<cell line: 3>:3 - Model loa

## Hyperparamètre

In [12]:
BATCH_SIZE = 32
LR=1e-4
PIN_MEMORY = True
NUM_WORKERS = 0
PREFETCH_FACTOR = 2
NUM_EPOCHS = 3
logger.info(f"{BATCH_SIZE=}")
logger.info(f"{LR=}")
logger.info(f"{PIN_MEMORY=}")
logger.info(f"{NUM_WORKERS=}")
logger.info(f"{PREFETCH_FACTOR=}")
logger.info(f"{NUM_EPOCHS=}")

2022-03-28 18:23:08.193 | INFO     | __main__:<cell line: 7>:7 - BATCH_SIZE=32
2022-03-28 18:23:08.195 | INFO     | __main__:<cell line: 8>:8 - LR=0.0001
2022-03-28 18:23:08.196 | INFO     | __main__:<cell line: 9>:9 - PIN_MEMORY=True
2022-03-28 18:23:08.198 | INFO     | __main__:<cell line: 10>:10 - NUM_WORKERS=0
2022-03-28 18:23:08.201 | INFO     | __main__:<cell line: 11>:11 - PREFETCH_FACTOR=2
2022-03-28 18:23:08.202 | INFO     | __main__:<cell line: 12>:12 - NUM_EPOCHS=1


In [13]:
train_dataset = JigsawDataset(train_df, tokenizer)
train_dataloader = DataLoader(train_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             num_workers=NUM_WORKERS,
                             prefetch_factor=PREFETCH_FACTOR,
                             pin_memory=PIN_MEMORY)

validation_dataset = JigsawDataset(validation_df, tokenizer)
validation_dataloader = DataLoader(validation_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             num_workers=NUM_WORKERS,
                             prefetch_factor=PREFETCH_FACTOR,
                             pin_memory=PIN_MEMORY)

# Pas besoin de Sigmoid en sorti du model seulement pour `BCEWithLogitsLoss`
criterion = torch.nn.BCELoss()
logger.info(criterion)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
logger.info(optimizer)

model.to(device)
criterion.to(device)

2022-03-28 18:23:08.545 | INFO     | __main__:<cell line: 19>:19 - BCELoss()
2022-03-28 18:23:08.550 | INFO     | __main__:<cell line: 21>:21 - AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    maximize: False
    weight_decay: 0.01
)


BCELoss()

## Metric

### Variantes de Hamming Loss

In [14]:
class HammingLossWithoutThreshold(Metric):
    def __init__(self, num_classes=1, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.num_classes = num_classes

        self.add_state("total", default=torch.tensor(0, dtype=torch.float32), dist_reduce_fx="sum")
        self.add_state("nbr_sample", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        current_nbr_sample, current_nbr_category = preds.shape
        if current_nbr_category != self.num_classes:
          raise AttributeError("`num_classes` != `current_nbr_category` detected in `pred` parameter")
        
        current_loss_per_pred = torch.absolute(target - preds)
        current_hamming_loss = current_loss_per_pred.sum()

        self.total += current_hamming_loss.float()
        self.nbr_sample += current_nbr_sample

    def compute(self):
        return self.total/(self.num_classes*self.nbr_sample)

In [15]:
class RebalancedHammingLossWithoutThreshold(Metric):
    def __init__(self, num_classes=1, average="macro", dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.num_classes = num_classes

        # average = "macro" or None
        self.average = average

        # Nombre de positif 1 & negatif 0 par categorie
        self.add_state(
            "number_positive",
            default=torch.tensor([0 for _ in range(num_classes)]),
            dist_reduce_fx="sum",
        )
        self.add_state(
            "number_negative",
            default=torch.tensor([0 for _ in range(num_classes)]),
            dist_reduce_fx="sum",
        )

        self.add_state(
            "hamming_loss_positive",
            default=torch.tensor([0.0 for _ in range(num_classes)]),
            dist_reduce_fx="sum",
        )
        self.add_state(
            "hamming_loss_negative",
            default=torch.tensor([0.0 for _ in range(num_classes)]),
            dist_reduce_fx="sum",
        )

        self.add_state("nbr_sample", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        current_nbr_sample, current_nbr_category = preds.shape
        if current_nbr_category != self.num_classes:
            raise AttributeError(
                "`num_classes` != `current_nbr_category` detected in `pred` parameter"
            )

        # Nombre de positif 1 & negatif 0 par categorie
        current_number_positive = target.sum(axis=0)
        current_number_negative = current_nbr_sample - target.sum(axis=0)

        self.number_positive += current_number_positive.int()
        self.number_negative += current_number_negative.int()

        self.nbr_sample += current_nbr_sample

        for class_id in range(self.num_classes):
            positive_filter = target[:, class_id] == 1
            negative_filter = target[:, class_id] == 0

            target_vector = target[:, class_id]
            preds_vector = preds[:, class_id]

            # Filtered vector
            ## Target
            pos_filtered_target_vector = target_vector[positive_filter]
            neg_filtered_target_vector = target_vector[negative_filter]
            ## Preds
            pos_filtered_preds_vector = preds_vector[positive_filter]
            neg_filtered_preds_vector = preds_vector[negative_filter]

            # Hamming Loss without Threshold
            hamming_loss_on_positive = torch.absolute(
                pos_filtered_target_vector - pos_filtered_preds_vector
            )
            hamming_loss_on_negative = torch.absolute(
                neg_filtered_target_vector - neg_filtered_preds_vector
            )

            self.hamming_loss_positive[class_id] += hamming_loss_on_positive.sum()
            self.hamming_loss_negative[class_id] += hamming_loss_on_negative.sum()

    def compute(self):
        factor_pos = self.nbr_sample / (2 * self.number_positive)
        factor_neg = self.nbr_sample / (2 * self.number_negative)

        rebalanced_hamming_loss_per_class = torch.multiply(
            self.hamming_loss_positive, factor_pos
        ) + torch.multiply(self.hamming_loss_negative, factor_neg)
        if self.average == "macro":
            return rebalanced_hamming_loss_per_class.sum() / (
                self.nbr_sample * self.num_classes
            )
        return rebalanced_hamming_loss_per_class / (self.nbr_sample)


### Instanciation des metrics

In [16]:
num_classes = len(LABEL_LIST)
train_metric_dict = dict()

# AUROC Macro
auroc_macro = AUROC(num_classes=num_classes, compute_on_step=True, average="macro")
train_metric_dict["auroc_macro"] = auroc_macro

# AUROC per class
auroc_per_class = AUROC(num_classes=num_classes, compute_on_step=True, average=None)
train_metric_dict["auroc_per_class"] = auroc_per_class

# F1 score global
f1 = F1Score()
train_metric_dict["f1"] = f1

# F1 score per class
f1_per_calss = F1Score(num_classes=6, average=None)
train_metric_dict["f1_per_calss"] = f1_per_calss

# Hamming Distance without Threshold
hamming_loss_woutt = HammingLossWithoutThreshold(num_classes=num_classes)
train_metric_dict["hamming_loss_without_threshold"] = hamming_loss_woutt

# Rebalanced Hamming Distance without Threshold macro
rebalanced_hamming_loss_woutt_macro = RebalancedHammingLossWithoutThreshold(
    num_classes=num_classes, average="macro"
)
train_metric_dict[
    "rebalanced_hamming_loss_without_threshold_macro"
] = rebalanced_hamming_loss_woutt_macro

# Rebalanced Hamming Distance without Threshold macro
rebalanced_hamming_loss_woutt_per_class = RebalancedHammingLossWithoutThreshold(
    num_classes=num_classes, average=None
)
train_metric_dict[
    "rebalanced_hamming_loss_without_threshold_per_class"
] = rebalanced_hamming_loss_woutt_per_class

In [17]:
train_metric = MetricCollection(train_metric_dict)
train_metric.to(device)

validation_metric = train_metric.clone()
validation_metric.to(device)

MetricCollection(
  (auroc_macro): AUROC()
  (auroc_per_class): AUROC()
  (f1): F1Score()
  (f1_per_calss): F1Score()
  (hamming_loss_without_threshold): HammingLossWithoutThreshold()
  (rebalanced_hamming_loss_without_threshold_macro): RebalancedHammingLossWithoutThreshold()
  (rebalanced_hamming_loss_without_threshold_per_class): RebalancedHammingLossWithoutThreshold()
)

### Export metrics

In [18]:
def serialize(object_to_serialize: Any, ensure_ascii: bool = True) -> str:
    """
    Serialize any object, i.e. convert an object to JSON
    Args:
        object_to_serialize (Any): The object to serialize
        ensure_ascii (bool, optional): If ensure_ascii is true (the default), the output is guaranteed to have all incoming non-ASCII characters escaped. If ensure_ascii is false, these characters will be output as-is. Defaults to True.
    Returns:
            str: string of serialized object (JSON)
    """

    def dumper(obj: Any) -> Union[str, Dict]:
        """
        Function called recursively by json.dumps to know how to serialize an object.
        For example, for datetime, we try to convert it to ISO format rather than
        retrieve the list of attributes defined in its object.
        Args:
            obj (Any): The object to serialize
        Returns:
            Union[str, Dict]: Serialized object
        """
        if isinstance(obj, torch.Tensor):
            return obj.cpu().numpy().tolist()
        elif hasattr(obj, "__dict__"):
            return obj.__dict__
        return str(obj)

    return json.dumps(object_to_serialize, default=dumper, ensure_ascii=ensure_ascii)

In [19]:
def export_metric(metric_collection, **kwargs):
    """
    Export MetricCollection to json file

    Args:
        metric_collection: MetricCollection
        **kwargs: field to add in json line
    """
    with open(METRIC_FILE_PATH, "a") as f:
        metric_collection_value = metric_collection.compute()
        metric_collection_value.update(kwargs)
        serialized_value = serialize(metric_collection_value)
        f.write(serialized_value)
        f.write("\n")
    logger.success("Metrics are exported !")

## Entraînement

In [20]:
def train_epoch(epoch_id=None):
    model.train()
    logger.info(f"START EPOCH {epoch_id=}")

    progress = tqdm(train_dataloader, desc='training batch...', leave=False)
    for batch_id, batch in enumerate(progress):
        if batch_id % 1_000 == 0:
            valid_epoch(epoch_id=epoch, batch_id=batch_id)
        
        logger.trace(f"{batch_id=}")
        token_list_batch = batch["ids"].to(device)
        attention_mask_batch = batch["mask"].to(device)
        label_batch = batch["labels"].to(device)

        # Reset gradient
        optimizer.zero_grad()

        # Predict
        prediction_batch = model(token_list_batch, attention_mask_batch)
        transformed_prediction_batch = prediction_batch.squeeze()

        # Loss
        loss = criterion(transformed_prediction_batch.to(torch.float32), label_batch.to(torch.float32))

        # Metrics
        train_metrics_collection_dict = train_metric(transformed_prediction_batch.to(torch.float32), label_batch.to(torch.int32))
        logger.trace(train_metrics_collection_dict)

        # Backprop
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        loss.backward()
        # gradient clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Update progress bar description
        progress_description = "Train Loss : {loss:.4f} - Train AUROC : {acc:.4f}"
        auroc_macro_value = float(train_metrics_collection_dict["auroc_macro"])
        progress_description = progress_description.format(loss=loss.item(), acc=auroc_macro_value)
        progress.set_description(progress_description)

    logger.info(f"END EPOCH {epoch_id=}")

In [34]:
train_dataloader = DataLoader(train_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             num_workers=NUM_WORKERS,
                             prefetch_factor=PREFETCH_FACTOR,
                             pin_memory=PIN_MEMORY,
                             drop_last=True)

ts = torch.Size([32, 6])
drop_last=True
progress = tqdm(train_dataloader, desc='training batch...', leave=False)
for batch in progress:
    label_batch = batch["labels"].to(device)
    if label_batch.shape != ts:
        print(label_batch.shape)
        break

training batch...:   0%|          | 0/3989 [00:00<?, ?it/s]

In [21]:
@torch.no_grad()
def valid_epoch(epoch_id=None, batch_id=None):
    model.eval()
    logger.info(f"START VALIDATION {epoch_id=}{batch_id=}")
    validation_metric.reset()

    loss_list = []
    prediction_list = torch.Tensor([])
    target_list = torch.Tensor([])


    progress = tqdm(validation_dataloader, desc="valid batch...", leave=False)
    for _, batch in enumerate(progress):
        
        token_list_batch = batch["ids"].to(device)
        attention_mask_batch = batch["mask"].to(device)
        label_batch = batch["labels"].to(device)

        # Predict
        prediction_batch = model(token_list_batch, attention_mask_batch)

        transformed_prediction_batch = prediction_batch.squeeze()

        # Loss
        loss = criterion(
            transformed_prediction_batch.to(torch.float32),
            label_batch.to(torch.float32),
        )
        loss_list.append(loss.item())
        prediction_list = torch.concat(
            [prediction_list, transformed_prediction_batch.cpu()]
        )
        target_list = torch.concat([target_list, label_batch.cpu()])

        # Metrics
        validation_metric(transformed_prediction_batch.to(torch.float32), label_batch.to(torch.int32))

    loss_mean = np.mean(loss_list)
    logger.trace(validation_metric.compute())
    logger.info(f"END VALIDATION {epoch_id=}{batch_id=}")
    export_metric(validation_metric, epoch_id=epoch_id, batch_id=batch_id, loss=loss_mean)

In [22]:
torch.cuda.empty_cache()
progress =  tqdm(range(1,NUM_EPOCHS+1), desc='training epoch...', leave=True)
for epoch in progress:
    # Train
    train_epoch(epoch_id=epoch)

    # Validation
    valid_epoch(epoch_id=epoch)

    # Save
    torch.save(model, MODEL_FILE_PATH)

training epoch...:   0%|          | 0/1 [00:00<?, ?it/s]

2022-03-28 18:23:13.491 | INFO     | __main__:train_epoch:3 - START EPOCH epoch_id=1


training batch...:   0%|          | 0/3990 [00:00<?, ?it/s]

2022-03-28 18:23:13.586 | INFO     | __main__:valid_epoch:4 - START VALIDATION epoch_id=1batch_id=0


valid batch...:   0%|          | 0/998 [00:00<?, ?it/s]

...

valid batch...:   0%|          | 0/998 [00:00<?, ?it/s]

2022-03-28 18:39:25.743 | INFO     | __main__:valid_epoch:40 - END VALIDATION epoch_id=1batch_id=2000
2022-03-28 18:39:25.744 | SUCCESS  | __main__:export_metric:15 - Metrics are exported !
2022-03-28 18:42:47.740 | INFO     | __main__:valid_epoch:4 - START VALIDATION epoch_id=1batch_id=3000


valid batch...:   0%|          | 0/998 [00:00<?, ?it/s]

2022-03-28 18:45:52.116 | INFO     | __main__:valid_epoch:40 - END VALIDATION epoch_id=1batch_id=3000
2022-03-28 18:45:52.117 | SUCCESS  | __main__:export_metric:15 - Metrics are exported !


ValueError: Using a target size (torch.Size([1, 6])) that is different to the input size (torch.Size([6])) is deprecated. Please ensure they have the same size.

## Evaluation

In [23]:
try:
    del train_df
    del validation_df
except NameError:
    logger.warning("Train DataFrame & Validation DataFrame already deleted")

In [24]:
test_df = pd.read_csv(TEST_DATASET_PATH, index_col=0)
test_df = test_df[~test_df.comment_text.isna()]

In [25]:
test_dataset = JigsawDataset(test_df, tokenizer)
test_dataloader = DataLoader(test_dataset,
                             batch_size=BATCH_SIZE)

In [26]:
@torch.no_grad()
def evaluation(model):
    model.eval()
    logger.info(f"START EVALUATION")

    index_tensor = torch.Tensor([])
    prediction_tensor = torch.Tensor([])

    progress = tqdm(test_dataloader, desc='test batch...', leave=False)
    for batch_id, batch in enumerate(progress):
        logger.trace(f"{batch_id=}")
        index_batch = batch["index"].to(device)
        token_list_batch = batch["ids"].to(device)
        attention_mask_batch = batch["mask"].to(device)
        label_batch = batch["labels"].to(device)

        # Predict
        prediction_batch = model(token_list_batch, attention_mask_batch)
        transformed_prediction_batch = prediction_batch.squeeze()
        
        index_tensor = torch.concat([index_tensor, index_batch.cpu()])
        prediction_tensor = torch.concat([prediction_tensor, transformed_prediction_batch.cpu()])
    
    logger.info(f"END EVALUATION")
    prediction_test_df = pd.DataFrame(prediction_tensor.tolist(), 
                                     columns=LABEL_LIST,
                                     index=index_tensor.to(int).tolist())
    prediction_test_df.to_csv(TEST_FILE_PATH)
    logger.success(f"Test predictions exported !")

In [27]:
evaluation(model)

2022-03-28 18:50:28.570 | INFO     | __main__:evaluation:4 - START EVALUATION


test batch...:   0%|          | 0/2000 [00:00<?, ?it/s]

ValueError: Input nan is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.